In [7]:
from matplotlib import pyplot as plt
import tensorflow as tf
import tensorcircuit as tc

K = tc.set_backend("tensorflow")

In [8]:
(x_train, y_train), (x_test, y_test) = tc.templates.dataset.mnist_pair_data(
    0, 1, loader=tf.keras.datasets.fashion_mnist
)

In [10]:
x_train = tf.image.pad_to_bounding_box(x_train, 2, 2, 32, 32)
x_test = tf.image.pad_to_bounding_box(x_test, 2, 2, 32, 32)

In [14]:
x_test.shape

TensorShape([2000, 32, 32, 1])

In [22]:
batched_ae = K.vmap(tc.templates.dataset.amplitude_encoding, vectorized_argnums=0)

In [23]:
x_train_q = batched_ae(x_train, 10)
x_test_q = batched_ae(x_test, 10)

In [10]:
n = 10
blocks = 3


def qml(x, weights):
    c = tc.Circuit(n, inputs=x)
    for j in range(blocks):
        for i in range(n):
            c.rx(i, theta=weights[j, i, 0])
            c.rz(i, theta=weights[j, i, 1])
        for i in range(n - 1):
            c.exp1(i, i + 1, theta=weights[j, i, 2], unitary=tc.gates._zz_matrix)
    outputs = K.stack(
        [K.real(c.expectation([tc.gates.z(), [i]])) for i in range(n)]
        + [K.real(c.expectation([tc.gates.x(), [i]])) for i in range(n)]
    )
    outputs = K.reshape(outputs, [-1])
    return K.sigmoid(K.sum(outputs))


qml_layer = tc.keras.QuantumLayer(qml, weights_shape=[blocks, n, 3])

In [12]:
model = tf.keras.Sequential([qml_layer])
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.legacy.Adam(0.01),
    metrics=[tf.keras.metrics.BinaryAccuracy()],
)

model.fit(x_train_q, y_train, batch_size=32, epochs=3, validation_split=0.8)

Epoch 1/3
75/75 [==============================] - 27s 318ms/step - loss: 0.5613 - binary_accuracy: 0.8003 - val_loss: 0.4014 - val_binary_accuracy: 0.9362
Epoch 2/3
75/75 [==============================] - 11s 150ms/step - loss: 0.3597 - binary_accuracy: 0.9312 - val_loss: 0.3093 - val_binary_accuracy: 0.9381
Epoch 3/3
75/75 [==============================] - 11s 144ms/step - loss: 0.2878 - binary_accuracy: 0.9337 - val_loss: 0.2576 - val_binary_accuracy: 0.9385
